In [37]:
import torch.nn as nn
import torch
import os
import cv2
import numpy as np
import torch.utils.data as utils

In [27]:
class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(3, 16, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(16, 8, 3, stride=2, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=1)
        )
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(8, 16, 3, stride=2),
            nn.ReLU(True),
            nn.ConvTranspose2d(16, 8, 5, stride=2, padding=1),
            nn.ReLU(True),
            nn.ConvTranspose2d(8, 3, 2, stride=2, padding=2, output_padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        return x

In [28]:
model_real = autoencoder()
model_synthetic = autoencoder()

In [29]:
model_real.load_state_dict(torch.load("./conv_autoencoder_real.pth"))
model_synthetic.load_state_dict(torch.load("./conv_autoencoder_synthetic.pth"))

In [30]:
model_real

autoencoder(
  (encoder): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (decoder): Sequential(
    (0): ConvTranspose2d(8, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): ReLU(inplace)
    (2): ConvTranspose2d(16, 8, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (3): ReLU(inplace)
    (4): ConvTranspose2d(8, 3, kernel_size=(2, 2), stride=(2, 2), padding=(2, 2), output_padding=(1, 1))
    (5): Tanh()
  )
)

In [15]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [16]:
images_synthetic = load_images_from_folder("./cvidea/validation_cvedia/Data")
images_real = load_images_from_folder("./cvidea/validation_flir/PreviewData")

In [19]:
images_real = [i.reshape(3,299,299) for i in images_real]
images_synthetic = [i.reshape(3,299,299) for i in images_synthetic]

images_real = [i/255.0 for i in images_real]
images_synthetic = [i/255.0 for i in images_synthetic]

images_real = np.array(images_real)
images_synthetic = np.array(images_synthetic)

In [35]:
tensor_real = torch.stack([torch.Tensor(i) for i in images_real])
tensor_synthetic = torch.stack([torch.Tensor(i) for i in images_synthetic])

In [38]:
dataset_real = utils.TensorDataset(tensor_real)
dataset_synthetic = utils.TensorDataset(tensor_synthetic)

In [39]:
dataloader_real = utils.DataLoader(dataset_real, batch_size=8, shuffle=True)
dataloader_synthetic = utils.DataLoader(dataset_synthetic, batch_size=8, shuffle=True)

In [40]:
ae_output_real = []
for data in dataloader_real:
    out = model_real(data[0])
    ae_output_real.append(out)

In [45]:
torch.save(ae_output_real, "real_output_encoder.pkl")

In [46]:
del ae_output_real

In [47]:
ae_output_synthetic = []
for data in dataloader_synthetic:
    out = model_synthetic(data[0])
    ae_output_synthetic.append(out)

In [48]:
torch.save(ae_output_synthetic, "synthetic_output_encoder.pkl")